In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
#from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

#from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

import optuna
from optuna.trial import Trial
from optuna.samplers import TPESampler
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Data Loading
X_model = pd.read_csv('X_model.csv')
Y_model = pd.read_csv('Y_model.csv')
# Define preprocessors
def preprocess(X: pd.DataFrame):
    X = X.fillna(0)
    # Create new dataframe from columns "gender", "age_code", and "region_code"
    X_new = pd.DataFrame(X, columns=['gender', 'age_code', 'region_code'])
    GIT_range = r"202205[0-9]{2}"
    cGIT = np.sum(X.filter(regex=("c" + GIT_range), axis=1).values, axis=1)
    tGIT = np.sum(X.filter(regex=("t" + GIT_range), axis=1).values, axis=1)
    sGIT = np.sum(X.filter(regex=("s" + GIT_range), axis=1).values, axis=1)

    VAT_range = r"20220[17](?:[01][0-9]|2[0-5])"
    tVAT = np.sum(X.filter(regex=("t" + VAT_range), axis=1).values, axis=1)
    sVAT = np.sum(X.filter(regex=("s" + VAT_range), axis=1).values, axis=1)

    entire_range = r"2022[0-9]{4}"
    cEntire = np.sum(X.filter(regex=("c" + entire_range), axis=1).values, axis=1)
    tEntire = np.sum(X.filter(regex=("t" + entire_range), axis=1).values, axis=1)
    sEntire = np.sum(X.filter(regex=("s" + entire_range), axis=1).values, axis=1)

    rcGIT = np.divide(cGIT, cEntire)
    rtGIT = np.divide(tGIT, tEntire)
    rsGIT = np.divide(sGIT, sEntire)

    rtVAT = np.divide(tVAT, tEntire)
    rsVAT = np.divide(sVAT, sEntire)

    X_new['cGIT'] = cGIT
    X_new['tGIT'] = tGIT
    X_new['sGIT'] = sGIT

    X_new['tVAT'] = tVAT
    X_new['sVAT'] = sVAT

    X_new['cEntire'] = cEntire
    X_new['tEntire'] = tEntire
    X_new['sEntire'] = sEntire

    X_new['rcGIT'] = rcGIT
    X_new['rtGIT'] = rtGIT
    X_new['rsGIT'] = rsGIT

    X_new['rtVAT'] = rtVAT
    X_new['rsVAT'] = rsVAT

    X_new = X_new.fillna(0)
    print(X_new.head())

    return X_new
# Preprocess
X_model = preprocess(X_model)

   gender  age_code  region_code  cGIT  tGIT    sGIT  tVAT    sVAT  cEntire  \
0       1        13            7   0.0   0.0     0.0   0.0     0.0      1.0   
1       1         5            1   2.0   0.0    17.0   0.0   185.0     39.0   
2       2         6            2   6.0   3.0  2253.0   0.0  1712.0     26.0   
3       2         1            1   2.0   0.0   342.0   0.0     0.0      5.0   
4       2         5            1   2.0   0.0   273.0   0.0   104.0     10.0   

   tEntire  sEntire     rcGIT  rtGIT     rsGIT  rtVAT     rsVAT  
0      1.0     93.0  0.000000    0.0  0.000000    0.0  0.000000  
1      0.0    790.0  0.051282    0.0  0.021519    0.0  0.234177  
2      3.0   5119.0  0.230769    1.0  0.440125    0.0  0.334440  
3      0.0    647.0  0.400000    0.0  0.528594    0.0  0.000000  
4      1.0    608.0  0.200000    0.0  0.449013    0.0  0.171053  


In [35]:
features = X_model[['cGIT', 'tGIT', 'sGIT', 'tVAT', 'sVAT', 'cEntire', 'tEntire', 'rtGIT', 'rtVAT']]
check = Y_model[['business']]

In [36]:
train_features, test_features, train_labels, test_labels = train_test_split(features, check)

In [37]:
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)
test_features = scaler.fit_transform(test_features)

In [38]:
model = LogisticRegression()
model.fit(train_features, train_labels)
tr_probs = model.predict_proba(train_features)
tr_probs = tr_probs[:, 1]
tr_auc = roc_auc_score(train_labels, tr_probs)
# summarize scores
print('Logistic: ROC AUC=%.3f' % (tr_auc))


Logistic: ROC AUC=0.792


In [39]:
lr_probs = model.predict_proba(test_features)
lr_probs = lr_probs[:, 1]
lr_auc = roc_auc_score(test_labels, lr_probs)
# summarize scores
print('Logistic: ROC AUC=%.3f' % (lr_auc))


Logistic: ROC AUC=0.789


In [40]:
print(model.score(train_features, train_labels))
print(model.score(test_features, test_labels))

0.9348716666666667
0.934155


In [33]:
print(model.predict_proba(test_features))
print(model.coef_)

[[0.90211082 0.09788918]
 [0.97381255 0.02618745]
 [0.97534645 0.02465355]
 ...
 [0.96758977 0.03241023]
 [0.97517868 0.02482132]
 [0.94384011 0.05615989]]
[[-0.10038064 -0.07545631 -0.09710461  0.16063041 -0.18079598  0.14201198
  -0.0023393   0.76219841  0.26446474  0.29365219]]


In [11]:
X_exam = pd.read_csv('X_exam.csv')
X_exam = preprocess(X_exam)

   gender  age_code  region_code  cGIT  tGIT   sGIT  tVAT    sVAT  cEntire  \
0       2         4            4   4.0   0.0   19.0   2.0   468.0     16.0   
1       2        10           15   0.0   0.0    0.0   0.0     0.0      7.0   
2       2         4            2   4.0   0.0  800.0   2.0  2719.0     48.0   
3       1         8            1   1.0   0.0   50.0   0.0     0.0     13.0   
4       2         8            2   0.0   0.0    0.0   1.0   488.0      1.0   

   tEntire  sEntire     rcGIT  rtGIT     rsGIT     rtVAT     rsVAT  
0      2.0   1187.0  0.250000    0.0  0.016007  1.000000  0.394271  
1      0.0   1253.0  0.000000    0.0  0.000000  0.000000  0.000000  
2      7.0   4502.0  0.083333    0.0  0.177699  0.285714  0.603954  
3      0.0    979.0  0.076923    0.0  0.051073  0.000000  0.000000  
4      1.0    488.0  0.000000    0.0  0.000000  1.000000  1.000000  


In [46]:
Y_exam = np.zeros(X_exam.shape[0])
k = 5
kf = StratifiedKFold(n_splits=k)
for tr_index, val_index in kf.split(X_model,Y_model):
    X_tr = X_model.iloc[tr_index]
    Y_tr = Y_model.iloc[tr_index]
    X_val, Y_val = X_model.iloc[val_index],Y_model.iloc[val_index]
    model.fit(X_val,Y_val)
    proba = model.predict_proba(X_exam)[:,1]
    Y_exam = Y_exam + proba
Y_exam = Y_exam/k
thresholds = np.array([0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4,0.45,0.5,0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95])
# the ratio of high prob with different thresholds
for num in thresholds: 
    filtered = Y_exam[np.where(Y_exam>num)]
    print("the number of probability more than %.2f is %d:" %(num,len(filtered)))
    print("the ratio of probability more than %.2f is : %.4f"%(num, float(len(filtered))/len(Y_exam)))
    print('---------------------------------------------------\n')

the number of probability more than 0.05 is 123596:
the ratio of probability more than 0.05 is : 0.6180
---------------------------------------------------

the number of probability more than 0.10 is 67921:
the ratio of probability more than 0.10 is : 0.3396
---------------------------------------------------

the number of probability more than 0.15 is 33331:
the ratio of probability more than 0.15 is : 0.1667
---------------------------------------------------

the number of probability more than 0.20 is 15481:
the ratio of probability more than 0.20 is : 0.0774
---------------------------------------------------

the number of probability more than 0.25 is 7475:
the ratio of probability more than 0.25 is : 0.0374
---------------------------------------------------

the number of probability more than 0.30 is 4410:
the ratio of probability more than 0.30 is : 0.0221
---------------------------------------------------

the number of probability more than 0.35 is 2915:
the ratio of pr

In [47]:
for th in [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4,0.45,0.5,0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]:
    profit = 0
    for tr_index, val_index in kf.split(X_model,Y_model):
        X_tr,Y_tr = X_model.iloc[tr_index],Y_model.iloc[tr_index]
        X_val,Y_val = X_model.iloc[val_index],Y_model.iloc[val_index]

        model.fit(X_tr,Y_tr)
        proba = model.predict_proba(X_val)[:,1]

        df = pd.DataFrame({'business_prob':proba},index=val_index)
        res = pd.concat([Y_val,df],axis=1)
        filtered = res[res['business_prob']>th]

        profit += filtered['business'].sum()* 500000*0.01 - 400*filtered.shape[0]
    # avg profit
    print("---------------------------------------------------------")
    print("when threshold is %.2f, the avg profit is %.2f" %(th,profit/5))
    print("---------------------------------------------------------")

---------------------------------------------------------
when threshold is 0.05, the avg profit is -7157160.00
---------------------------------------------------------
---------------------------------------------------------
when threshold is 0.10, the avg profit is -3810960.00
---------------------------------------------------------
---------------------------------------------------------
when threshold is 0.15, the avg profit is 605280.00
---------------------------------------------------------
---------------------------------------------------------
when threshold is 0.20, the avg profit is 3035920.00
---------------------------------------------------------
---------------------------------------------------------
when threshold is 0.25, the avg profit is 4000920.00
---------------------------------------------------------
---------------------------------------------------------
when threshold is 0.30, the avg profit is 3877880.00
-------------------------------------------